![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## This notebook is an example of how to pipeline two models. 
A video stream is processed by the person detection model. The person detection results are then processed by the pose detection model, one person bounding box at a time. Combined result is then displayed.

This example uses `degirum_tools.streams` streaming toolkit.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

The script can use either a web camera or local camera connected to the machine or a video file. The camera index or URL or video file path needs to be specified in the code below by assigning `video_source`.

In [1]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences, model_zoo_url, model names for inference, and video source

In [9]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# people_det_model_name: name of the model for detecting people
# pose_det_model_name: name of the model for pose detection
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
hw_location = "@cloud"
model_zoo_url = "degirum/public"
people_det_model_name = "yolo_v5s_person_det--512x512_quant_n2x_orca1_1"
pose_det_model_name = "mobilenet_v1_posenet_coco_keypoints--353x481_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/WalkingPeople.mp4"

#### The rest of the cells below should run without any modifications

In [12]:
import degirum as dg, degirum_tools
from degirum_tools import streams as dgstreams

# connect to AI inference engine
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load person detection model
person_det_model = zoo.load_model(
    people_det_model_name,
    output_confidence_threshold=0.7,
    overlay_show_probabilities=True,
    overlay_line_width=1,
)

# load pose detection model
pose_det_model = zoo.load_model(
    pose_det_model_name,
    output_pose_threshold=0.2,
    overlay_line_width=1,
    overlay_show_labels=False,
    overlay_color=(255, 0, 0),
)

In [ ]:
# create gizmos

source = dgstreams.VideoSourceGizmo(video_source)  # video source
person = dgstreams.AiSimpleGizmo(person_det_model)  # person detector
crop = dgstreams.AiObjectDetectionCroppingGizmo(["person"])  # cropping gizmo
pose = dgstreams.AiSimpleGizmo(pose_det_model)  # pose detector
combiner = dgstreams.CropCombiningGizmo()  # combiner
display = dgstreams.VideoDisplayGizmo(
    "Poses", show_ai_overlay=True, show_fps=True
)  # display

# create pipeline and composition, then start it
dgstreams.Composition(
    source >> person >> crop >> pose >> combiner[1] >> display, person >> combiner[0]
).start()